In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import re
import numpy as np
from keras.utils import np_utils
from PIL import Image
from numpy import asarray

Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split
from keras import backend as K
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Input,Dense,Activation,Flatten,Dropout,Convolution2D,MaxPooling2D,Lambda
from tensorflow.python.keras.optimizers import RMSprop

In [0]:
#Reads image and returns a numpy array

def read_image(filename):
  image=Image.open(filename)
  data=asarray(image)
  data.shape

  return data

In [0]:
#testing the function
image1=read_image('/content/drive/My Drive/s1/1.pgm')
image1.shape

(112, 92)

In [0]:
size = 2
total_sample_size = 10000

def get_data(size, total_sample_size):
    #read the image
    image = read_image('/content/drive/My Drive/s' + str(1) + '/' + str(1) + '.pgm')
    #reduce the size
    image = image[::size, ::size]
    #get the new size
    dim1 = image.shape[0]
    dim2 = image.shape[1]

    count = 0
    
    #initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
    x_geuine_pair = np.zeros([total_sample_size, 2, dim1, dim2,1]) # 2 is for pairs
    y_genuine = np.zeros([total_sample_size, 1])
    
    for i in range(40):
        for j in range(int(total_sample_size/40)):
            ind1 = 0
            ind2 = 0
            
            #read different images from same directory (genuine pair)
            while ind1 == ind2:
                ind1 = np.random.randint(10)
                ind2 = np.random.randint(10)
            
            # read the two images
            img1 = read_image('/content/drive/My Drive/s' + str(i+1) + '/' + str(ind1 + 1) + '.pgm')
            img2 = read_image('/content/drive/My Drive/s' + str(i+1) + '/' + str(ind2 + 1) + '.pgm')
            
            #reduce the size
            img1 = img1[::size, ::size]
            img2 = img2[::size, ::size]
            
            #store the images to the initialized numpy array
            x_geuine_pair[count, 0, :, :, 0] = img1
            x_geuine_pair[count, 1, :, :, 0] = img2
            
            #as we are drawing images from the same directory we assign label as 1. (genuine pair)
            y_genuine[count] = 1
            count += 1

    # print(x_geuine_pair)
    count = 0
    x_imposite_pair = np.zeros([total_sample_size, 2, dim1, dim2, 1])
    y_imposite = np.zeros([total_sample_size, 1])
    
    for i in range(int(total_sample_size/10)):
        for j in range(10):
            
            #read images from different directory (imposite pair)
            while True:
                ind1 = np.random.randint(40)
                ind2 = np.random.randint(40)
                if ind1 != ind2:
                    break
                    
            img1 = read_image('/content/drive/My Drive/s' + str(ind1+1) + '/' + str(j + 1) + '.pgm')
            img2 = read_image('/content/drive/My Drive/s' + str(ind2+1) + '/' + str(j + 1) + '.pgm')

            img1 = img1[::size, ::size]
            img2 = img2[::size, ::size]

            x_imposite_pair[count, 0, :, :, 0] = img1
            x_imposite_pair[count, 1, :, :, 0] = img2
            #as we are drawing images from the different directory we assign label as 0. (imposite pair)
            y_imposite[count] = 0
            count += 1
    
    # print(x_imposite_pair)

    #now, concatenate, genuine pairs and imposite pair to get the whole data
    X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_imposite], axis=0)

    return X, Y

In [0]:
X, Y = get_data(size, total_sample_size)

In [0]:
X.shape

(20000, 2, 56, 46, 1)

In [0]:
#splitting into train and test data
X_train,X_test,Y_train,Y_test =train_test_split(X,Y,test_size=0.25)

In [0]:
def siamese_network(input_shape):

  model=Sequential()

  model.add(Convolution2D(6,(3,3),input_shape=input_shape,activation='relu',padding='valid'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.25))

  model.add(Convolution2D(12,(3,3),activation='relu',padding='valid'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(128,activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(25,activation='relu'))

  return model



In [0]:
input_dim=X_train.shape[2:]
img_a=Input(shape=input_dim)
img_b=Input(shape=input_dim)

In [0]:
# import tensorflow as tf
# tf.__version__

'2.2.0-rc2'

In [0]:
input_dim
# images_nchw = tf.placeholder(tf.float32, input_dim)  # input batch
# out = tf.transpose(x, [0, 2, 3, 1])

(56, 46, 1)

In [0]:
network=siamese_network(input_dim)
feature_a=network(img_a)
feature_b=network(img_b)

In [0]:
#Defining the distance functions

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

In [0]:
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [0]:
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feature_a, feature_b])

In [0]:
out=Dense(2,activation='softmax')(distance)

In [0]:
seq = Model(inputs=[img_a,img_b], outputs=out)

In [0]:
seq.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 56, 46, 1)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 56, 46, 1)]  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 25)           188393      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           sequential[1][0]             

In [0]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

seq.compile(loss=contrastive_loss, optimizer='rmsprop')

In [0]:
Y_train=np_utils.to_categorical(Y_train,2)
Y_test=np_utils.to_categorical(Y_test,2)

In [0]:
# Y_train

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [0]:
img_1 = X_train[:, 0]
img_2 = X_train[:, 1] 

# X_train[:,1].shape[0]

In [0]:
seq.fit([img_1, img_2], Y_train, validation_split=.25, batch_size=128, verbose=1, epochs=100)

Epoch 1/100


InvalidArgumentError: ignored

In [0]:
img_ta=X_test[:,0]
img_tb=X_test[:,1]

# Y_test=np

In [0]:
score=seq.evaluate([img_ta,img_tb],Y_test[:],verbose=1)
# print("Accuracy ",score[1])


157/157 [==============================] - 0s 3ms/step - loss: 0.4824


In [0]:
print("Accuracy ",1-score)

Accuracy  0.5176154971122742


In [0]:
pred = seq.predict([X_test[:, 0], X_test[:, 1]])

In [0]:
pred.shape

(5000, 1)